# Clasificador de Fraude en Canastas de Compra

Este notebook implementa un clasificador binario para detectar fraude en canastas de compra utilizando Regresión Logística.

**Problema**: Predecir si una canasta de compra es fraudulenta basándose en los artículos comprados y características agregadas.

**Dataset**: 9,318 canastas con 2,474 productos + 6 características agregadas

**Modelo principal**: Regresión Logística con regularización Ridge (similar al problema de detección de spam)

# 1. Bibliotecas y Funciones

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    classification_report,
    roc_curve, 
    auc,
    RocCurveDisplay
)

import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
def evaluacion_modelo(modelo, X_train, y_train, X_test, y_test, nombre_modelo='Modelo'):
    """
    Evalúa el modelo en conjuntos de entrenamiento y prueba.
    Muestra matrices de confusión lado a lado.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Evaluación en entrenamiento
    y_train_pred = modelo.predict(X_train)
    acc_train = accuracy_score(y_train, y_train_pred) * 100
    
    cm_train = pd.crosstab(
        y_train, 
        y_train_pred,
        rownames=['Real'],
        colnames=['Predicción']
    )
    sns.heatmap(cm_train, annot=True, fmt='g', ax=ax1, cmap='Blues')
    ax1.set_title(f'{nombre_modelo} - Entrenamiento\nAccuracy: {acc_train:.2f}%')
    
    # Evaluación en prueba
    y_test_pred = modelo.predict(X_test)
    acc_test = accuracy_score(y_test, y_test_pred) * 100
    
    cm_test = pd.crosstab(
        y_test,
        y_test_pred,
        rownames=['Real'],
        colnames=['Predicción']
    )
    sns.heatmap(cm_test, annot=True, fmt='g', ax=ax2, cmap='Blues')
    ax2.set_title(f'{nombre_modelo} - Prueba\nAccuracy: {acc_test:.2f}%')
    
    plt.tight_layout()
    plt.show()
    
    # Reporte de clasificación
    print(f"\n{'='*60}")
    print(f"Reporte de Clasificación - {nombre_modelo} (Conjunto de Prueba)")
    print(f"{'='*60}")
    print(classification_report(y_test, y_test_pred, target_names=['No Fraude', 'Fraude']))
    
    return acc_train, acc_test

In [3]:
def matriz_confusion_con_umbral(modelo, X_test, y_test, umbral=0.5):
    """
    Crea matriz de confusión usando un umbral personalizado.
    """
    y_pred_proba = modelo.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > umbral).astype(int)
    
    cm = pd.crosstab(
        y_test,
        y_pred,
        rownames=['Real'],
        colnames=['Predicción']
    )
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.title(f'Matriz de Confusión (umbral={umbral})')
    plt.show()
    
    acc = accuracy_score(y_test, y_pred) * 100
    print(f"\nAccuracy con umbral {umbral}: {acc:.2f}%")
    print(f"\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred, target_names=['No Fraude', 'Fraude']))

# 2. Lectura y Exploración Inicial de Datos

In [8]:
# Cargar datos
df = pd.read_csv('/Users/andrestrevino/Bourbaki/Working_Analyst/Examen_reto1_clasificador_canastas_fraude/Datos/FraudeCanastas.csv')

print(f"Dimensiones del dataset: {df.shape}")
print(f"\nPrimeras filas:")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrestrevino/Bourbaki/Working_Analyst/Examen_reto1_clasificador_canastas_fraude/Datos/FraudeCanastas.csv'

In [ ]:
# Información general
print("Información del dataset:")
print(f"Total de registros: {len(df)}")
print(f"Total de columnas: {df.shape[1]}")
print(f"\nValores nulos por columna:")
print(df.isnull().sum().sum())  # Total de nulos

In [ ]:
# Análisis de la variable objetivo
print("Distribución de la variable objetivo (fraud_flag):")
print(df['fraud_flag'].value_counts())
print(f"\nProporción:")
print(df['fraud_flag'].value_counts(normalize=True))

# Visualizar distribución de clases
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Conteo
df['fraud_flag'].value_counts().plot(kind='bar', ax=ax[0], color=['green', 'red'])
ax[0].set_title('Distribución de Clases')
ax[0].set_xlabel('Fraude (0=No, 1=Sí)')
ax[0].set_ylabel('Frecuencia')
ax[0].set_xticklabels(['No Fraude', 'Fraude'], rotation=0)

# Proporción
df['fraud_flag'].value_counts(normalize=True).plot(kind='pie', ax=ax[1], 
                                                    autopct='%1.1f%%',
                                                    colors=['green', 'red'],
                                                    labels=['No Fraude', 'Fraude'])
ax[1].set_title('Proporción de Clases')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

print(f"\n⚠️ Dataset DESBALANCEADO: {df['fraud_flag'].value_counts(normalize=True)[0]*100:.1f}% No Fraude vs {df['fraud_flag'].value_counts(normalize=True)[1]*100:.1f}% Fraude")

In [ ]:
# Identificar las columnas de características agregadas
caracteristicas_agregadas = ['Nb_of_items', 'total_of_items', 'costo_total', 
                              'costo_medio_item', 'costo_item_max', 'costo_item_min']

print("Características agregadas:")
df[caracteristicas_agregadas].describe()

In [ ]:
# Identificar columnas de productos (todas excepto ID, características agregadas y fraud_flag)
columnas_productos = [col for col in df.columns 
                      if col not in ['ID'] + caracteristicas_agregadas + ['fraud_flag']]

print(f"Total de columnas de productos: {len(columnas_productos)}")
print(f"\nPrimeros 10 productos:")
print(columnas_productos[:10])

In [ ]:
# Analizar sparsity de los datos de productos
productos_df = df[columnas_productos]
total_valores = productos_df.size
valores_cero = (productos_df == 0).sum().sum()
sparsity = (valores_cero / total_valores) * 100

print(f"Análisis de Dispersión (Sparsity):")
print(f"Total de valores: {total_valores:,}")
print(f"Valores en cero: {valores_cero:,}")
print(f"Sparsity: {sparsity:.2f}%")
print(f"\n💡 Los datos son muy dispersos (sparse), similar al problema de Bag-of-Words")

# 3. División Train/Test (TEMPRANA - Prevenir Data Leakage)

In [ ]:
# Separar características (X) y etiqueta (y)
X = df.drop(['ID', 'fraud_flag'], axis=1)
y = df['fraud_flag']

# División estratificada para mantener proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    shuffle=True,
    stratify=y,
    random_state=42
)

print(f"Conjunto de Entrenamiento:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  Proporción de fraude: {y_train.mean():.3f}")

print(f"\nConjunto de Prueba:")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"  Proporción de fraude: {y_test.mean():.3f}")

# 4. Exploración de Datos en Conjunto de Entrenamiento

In [ ]:
# Crear DataFrame temporal con train data para exploración
df_train = pd.concat([X_train, y_train], axis=1)

# Análisis univariado de características agregadas por clase
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(caracteristicas_agregadas):
    df_train.groupby('fraud_flag')[col].hist(alpha=0.6, bins=30, ax=axes[i], density=True)
    axes[i].set_title(f'{col}')
    axes[i].set_xlabel(col)
    axes[i].legend(['No Fraude', 'Fraude'])

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots de características agregadas por clase
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(caracteristicas_agregadas):
    sns.boxplot(x='fraud_flag', y=col, data=df_train, ax=axes[i], palette='Set2')
    axes[i].set_title(f'{col} por clase')
    axes[i].set_xticklabels(['No Fraude', 'Fraude'])

plt.tight_layout()
plt.show()

In [ ]:
# Estadísticas descriptivas por clase
print("Estadísticas de características agregadas por clase:\n")
print("NO FRAUDE (0):")
print(df_train[df_train['fraud_flag']==0][caracteristicas_agregadas].describe())
print("\n" + "="*80 + "\n")
print("FRAUDE (1):")
print(df_train[df_train['fraud_flag']==1][caracteristicas_agregadas].describe())

In [ ]:
# Matriz de correlación de características agregadas
plt.figure(figsize=(10, 8))
correlacion = df_train[caracteristicas_agregadas + ['fraud_flag']].corr()
sns.heatmap(correlacion, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matriz de Correlación - Características Agregadas y Fraude')
plt.tight_layout()
plt.show()

print("\nCorrelación con fraud_flag:")
print(correlacion['fraud_flag'].sort_values(ascending=False))

In [ ]:
# Productos más frecuentes en fraude vs no-fraude
productos_fraude = df_train[df_train['fraud_flag']==1][columnas_productos]
productos_no_fraude = df_train[df_train['fraud_flag']==0][columnas_productos]

# Top 10 productos en canastas fraudulentas
top_fraude = (productos_fraude > 0).sum().sort_values(ascending=False).head(10)
print("Top 10 productos más frecuentes en FRAUDE:")
print(top_fraude)

print("\n" + "="*80 + "\n")

# Top 10 productos en canastas no fraudulentas
top_no_fraude = (productos_no_fraude > 0).sum().sort_values(ascending=False).head(10)
print("Top 10 productos más frecuentes en NO FRAUDE:")
print(top_no_fraude)

# 5. Modelo 1: Regresión Logística con Regularización Ridge

## 5.1 Modelo Base sin Regularización

In [ ]:
# Entrenar modelo base
modelo_base = LogisticRegression(
    penalty=None,
    max_iter=1000,
    random_state=42,
    solver='saga'
).fit(X_train, y_train)

# Evaluar
acc_train_base, acc_test_base = evaluacion_modelo(
    modelo_base, X_train, y_train, X_test, y_test, 
    nombre_modelo='Regresión Logística (Sin Regularización)'
)

## 5.2 Grid Search para encontrar mejor C (Ridge - L2)

In [ ]:
%%time

# Valores de lambda y C (C = 1/lambda)
lambdas = np.array([0.001, 0.01, 0.1, 1, 10, 100])
C_values = 1 / lambdas

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search
modelo_ridge = LogisticRegression(
    penalty='l2',
    max_iter=1000,
    solver='saga',
    random_state=42
)

grid_search = GridSearchCV(
    estimator=modelo_ridge,
    param_grid={'C': C_values},
    cv=kf,
    scoring='accuracy',
    return_train_score=True,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\n{'='*60}")
print(f"Mejor parámetro C: {grid_search.best_params_['C']}")
print(f"Mejor score (CV): {grid_search.best_score_:.4f}")
print(f"{'='*60}")

In [ ]:
# Resultados del Grid Search
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results_display = cv_results[['param_C', 'mean_train_score', 'mean_test_score']]
print("Resultados de Grid Search:")
print(cv_results_display)

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(cv_results['param_C'], cv_results['mean_train_score'], marker='o', label='Train Score')
plt.plot(cv_results['param_C'], cv_results['mean_test_score'], marker='s', label='CV Score')
plt.xscale('log')
plt.xlabel('C (1/lambda)')
plt.ylabel('Accuracy')
plt.title('Grid Search: Accuracy vs C')
plt.legend()
plt.grid(True)
plt.show()

## 5.3 Entrenar modelo final con mejor C

In [ ]:
# Mejor C
best_C = grid_search.best_params_['C']

# Entrenar modelo final
modelo_ridge_final = LogisticRegression(
    penalty='l2',
    C=best_C,
    max_iter=1000,
    solver='saga',
    random_state=42
).fit(X_train, y_train)

# Evaluar
acc_train_ridge, acc_test_ridge = evaluacion_modelo(
    modelo_ridge_final, X_train, y_train, X_test, y_test,
    nombre_modelo=f'Regresión Logística Ridge (C={best_C})'
)

## 5.4 Curva ROC

In [ ]:
# Curva ROC
RocCurveDisplay.from_estimator(modelo_ridge_final, X_test, y_test)
plt.title('Curva ROC - Regresión Logística Ridge')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.legend()
plt.grid(True)
plt.show()

# 6. Ajuste de Umbral para Clases Desbalanceadas

In [ ]:
# Analizar probabilidades predichas
y_pred_proba = modelo_ridge_final.predict_proba(X_test)[:, 1]

# DataFrame con probabilidades
resultados = pd.DataFrame({
    'P(Fraude)': y_pred_proba,
    'y_real': y_test.values,
    'y_pred_05': (y_pred_proba > 0.5).astype(int)
})

# Distribución de probabilidades
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
resultados[resultados['y_real']==0]['P(Fraude)'].hist(bins=50, alpha=0.6, label='No Fraude', color='green')
resultados[resultados['y_real']==1]['P(Fraude)'].hist(bins=50, alpha=0.6, label='Fraude', color='red')
plt.xlabel('P(Fraude)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Probabilidades Predichas')
plt.legend()
plt.axvline(0.5, color='black', linestyle='--', label='Umbral=0.5')

plt.subplot(1, 2, 2)
resultados.sort_values('P(Fraude)', ascending=False).head(100)['P(Fraude)'].plot()
plt.xlabel('Top 100 canastas')
plt.ylabel('P(Fraude)')
plt.title('Top 100 Probabilidades más altas')
plt.axhline(0.5, color='black', linestyle='--')
plt.grid(True)

plt.tight_layout()
plt.show()

print("Estadísticas de P(Fraude) por clase real:")
print(resultados.groupby('y_real')['P(Fraude)'].describe())

## 6.1 Probar diferentes umbrales

In [ ]:
# Umbral = 0.3 (más sensible al fraude)
print("\n" + "="*60)
print("UMBRAL = 0.3")
print("="*60)
matriz_confusion_con_umbral(modelo_ridge_final, X_test, y_test, umbral=0.3)

In [ ]:
# Umbral = 0.2 (aún más sensible)
print("\n" + "="*60)
print("UMBRAL = 0.2")
print("="*60)
matriz_confusion_con_umbral(modelo_ridge_final, X_test, y_test, umbral=0.2)

In [ ]:
# Análisis de trade-offs con diferentes umbrales
umbrales = np.arange(0.1, 0.9, 0.05)
metricas = []

for umbral in umbrales:
    y_pred = (y_pred_proba > umbral).astype(int)
    
    # Calcular métricas
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    metricas.append({
        'umbral': umbral,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })

metricas_df = pd.DataFrame(metricas)

# Graficar
plt.figure(figsize=(12, 6))
plt.plot(metricas_df['umbral'], metricas_df['accuracy'], marker='o', label='Accuracy')
plt.plot(metricas_df['umbral'], metricas_df['precision'], marker='s', label='Precision')
plt.plot(metricas_df['umbral'], metricas_df['recall'], marker='^', label='Recall')
plt.plot(metricas_df['umbral'], metricas_df['f1'], marker='d', label='F1-Score')
plt.xlabel('Umbral')
plt.ylabel('Score')
plt.title('Métricas vs Umbral de Decisión')
plt.legend()
plt.grid(True)
plt.axvline(0.5, color='red', linestyle='--', alpha=0.5, label='Umbral default')
plt.show()

print("\nTop 5 umbrales por F1-Score:")
print(metricas_df.sort_values('f1', ascending=False).head())

# 7. Análisis de Coeficientes e Importancia de Features

In [ ]:
# Obtener coeficientes
coeficientes = pd.Series(
    modelo_ridge_final.coef_[0],
    index=X_train.columns
).sort_values(ascending=False)

print("Top 20 características que MÁS predicen FRAUDE (coeficientes positivos):")
print(coeficientes.head(20))

print("\n" + "="*80 + "\n")

print("Top 20 características que MÁS predicen NO FRAUDE (coeficientes negativos):")
print(coeficientes.tail(20))

In [ ]:
# Visualizar top 15 coeficientes positivos y negativos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Top indicadores de fraude
coeficientes.head(15).sort_values().plot(kind='barh', ax=ax1, color='red')
ax1.set_title('Top 15 Indicadores de FRAUDE\n(Coeficientes positivos más altos)')
ax1.set_xlabel('Coeficiente')
ax1.axvline(0, color='black', linestyle='--', linewidth=0.5)

# Top indicadores de no-fraude
coeficientes.tail(15).sort_values().plot(kind='barh', ax=ax2, color='green')
ax2.set_title('Top 15 Indicadores de NO FRAUDE\n(Coeficientes negativos más bajos)')
ax2.set_xlabel('Coeficiente')
ax2.axvline(0, color='black', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de características agregadas
coefs_agregadas = coeficientes[caracteristicas_agregadas].sort_values(ascending=False)

plt.figure(figsize=(10, 6))
coefs_agregadas.plot(kind='barh', color=['red' if x > 0 else 'green' for x in coefs_agregadas])
plt.title('Coeficientes de Características Agregadas')
plt.xlabel('Coeficiente')
plt.axvline(0, color='black', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

print("Coeficientes de características agregadas:")
print(coefs_agregadas)

In [ ]:
# Odds Ratio (e^coeficiente)
# Indica cuántas veces aumenta/disminuye la probabilidad de fraude
odds_ratios = np.exp(coeficientes).sort_values(ascending=False)

print("Top 10 Odds Ratios (productos que MÁS aumentan prob. de fraude):")
print(odds_ratios.head(10))
print("\nInterpretación: Odds Ratio > 1 aumenta prob. de fraude")
print("               Odds Ratio < 1 disminuye prob. de fraude")

# 8. Modelo 2 (Opcional): Random Forest para Comparación

In [ ]:
%%time

# Entrenar Random Forest
modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # Para manejar desbalanceo
).fit(X_train, y_train)

print("Modelo Random Forest entrenado.")

In [ ]:
# Evaluar Random Forest
acc_train_rf, acc_test_rf = evaluacion_modelo(
    modelo_rf, X_train, y_train, X_test, y_test,
    nombre_modelo='Random Forest'
)

In [ ]:
# Feature Importance de Random Forest
importances = pd.Series(
    modelo_rf.feature_importances_,
    index=X_train.columns
).sort_values(ascending=False)

print("Top 20 Features más importantes (Random Forest):")
print(importances.head(20))

# Graficar
plt.figure(figsize=(10, 8))
importances.head(20).sort_values().plot(kind='barh', color='steelblue')
plt.title('Top 20 Features - Random Forest')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

## 8.1 Comparación de Modelos

In [ ]:
# Comparar curvas ROC
fig, ax = plt.subplots(figsize=(10, 8))

# Regresión Logística Ridge
RocCurveDisplay.from_estimator(
    modelo_ridge_final, X_test, y_test, 
    name=f'Logistic Regression Ridge (C={best_C})',
    ax=ax
)

# Random Forest
RocCurveDisplay.from_estimator(
    modelo_rf, X_test, y_test,
    name='Random Forest',
    ax=ax
)

# Baseline
ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

ax.set_title('Comparación de Curvas ROC')
ax.legend()
ax.grid(True)
plt.show()

In [ ]:
# Tabla comparativa
comparacion = pd.DataFrame({
    'Modelo': ['Logistic Regression (sin reg)', 'Logistic Regression Ridge', 'Random Forest'],
    'Accuracy Train': [acc_train_base, acc_train_ridge, acc_train_rf],
    'Accuracy Test': [acc_test_base, acc_test_ridge, acc_test_rf]
})

print("\nComparación de Modelos:")
print(comparacion)

# Graficar comparación
comparacion.set_index('Modelo')[['Accuracy Train', 'Accuracy Test']].plot(kind='bar', figsize=(10, 6))
plt.title('Comparación de Accuracy: Train vs Test')
plt.ylabel('Accuracy (%)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# 9. Conclusiones y Recomendaciones

## Resumen de Resultados:

### Dataset:
- **9,318 canastas** de compra
- **Clases desbalanceadas**: 86% No Fraude, 14% Fraude
- **Alta dimensionalidad**: 2,474 productos (datos muy dispersos/sparse)
- **6 características agregadas** numéricas

### Modelos Evaluados:
1. **Regresión Logística sin regularización**
2. **Regresión Logística con Ridge (L2)** ← Modelo recomendado
3. **Random Forest** (comparación)

### Mejor Modelo:
**Regresión Logística con Ridge** demostró ser el modelo más apropiado porque:
- Excelente desempeño con datos sparse de alta dimensionalidad
- Coeficientes interpretables
- Rápido entrenamiento
- Puede ajustar umbral para optimizar detección de fraude

### Patrones de Fraude Descubiertos:
- Ver sección 7 para productos específicos que indican fraude
- Características agregadas importantes: [analizar coeficientes]

### Recomendaciones:
1. **Ajustar umbral de decisión** según costo de falsos negativos (no detectar fraude)
2. **Monitorear productos con coeficientes positivos altos** (fuerte indicador de fraude)
3. **Considerar características adicionales** si están disponibles (hora de compra, ubicación, etc.)
4. **Re-entrenar modelo periódicamente** con nuevos datos para capturar patrones emergentes

### Métricas Clave:
- Umbral óptimo: [depende del análisis en sección 6]
- Recall (captura de fraudes): [ver matriz de confusión]
- Precision (minimizar falsos positivos): [ver matriz de confusión]
- AUC-ROC: [ver curva ROC]
